In [1]:
import numpy as np
import pandas as pd
import glob
import nltk 
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
import shutil
import random
from wordcloud import WordCloud
import gensim
import seaborn as sns
from textblob import TextBlob
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import FastText
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from scipy.sparse import hstack


nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ishaq\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishaq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishaq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
train_dff = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

In [3]:
train_df=train_dff[:]

In [4]:
def remove_punctuation(text):
    # Define the pattern to match punctuation
    punctuation_pattern = r'[^\w\s]'
    # Replace punctuation with an empty string
    text_without_punctuation = re.sub(punctuation_pattern, '', text)
    # Normalize whitespace
    normalized_text = re.sub(r'\s+', ' ', text_without_punctuation)
    return normalized_text

# Text Processing
def preprocess_text(text, method='snowballstemming'):
    if not isinstance(text, str):
        return ''
    # Tokenize and lowercase
    tokens = word_tokenize(text.lower())

    # Define the set of stopwords
    stop_words = {'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
                "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he',
                'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its',
                'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
                'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 
                'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 
                'but', 'if','or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
                'against', 'between', 'into', 'through','during', 'before', 'after', 'above', 'below', 'to', 'from',
                'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 
                'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both','each', 'few', 'more', 'most', 'other',
                'some', 'such','product', 'purchase', 'buy', 'bought', 'ordered', 'order', 'item', 'items', 
                'review', 'reviews', 'star', 'stars', 'rating', 'ratings', 'customer', 'customers', 
                'seller', 'sellers', 'seller', 'brand', 'brands', 'price', 'prices', 'money', 
                'value', 'quality', 'deal', 'deals','shipping', 'shipping', 'delivery', 'delivered', 'package', 
                'packaging','customer service', 'service', 'support', 'warranty', 'warranties', 
                'problem','use','one','work','machine','using','need','needed','cut','recieve',
                'glitter','pen','marker','folder','card', 'look','looking','color','background',
                'tool','design','made','craft','piece','store','size','markers','page','make','used' }


    tokens = [remove_punctuation(token) for token in tokens if token.isalnum() and token not in stop_words]
    
    if method == 'snowballstemming':
        stemmer = nltk.stem.SnowballStemmer('english')
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'porterstemming':
        stemmer = nltk.stem.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'lemmatization':
        lemmatizer = nltk.stem.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [5]:
train_df.dropna(subset='Review',inplace=True)
train_df['ssprocessing_text'] = train_df['Review'].apply(preprocess_text, method='snowballstemming')
# train_df['psprocessing_text'] = train_df['Review'].apply(preprocess_text, method='porterstemming')
# train_df['lemprocessing_text'] = train_df['Review'].apply(preprocess_text, method='lemmatization')

C:\Users\ishaq\AppData\Local\Temp\ipykernel_10648\3045761950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset='Review',inplace=True)
C:\Users\ishaq\AppData\Local\Temp\ipykernel_10648\3045761950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['ssprocessing_text'] = train_df['Review'].apply(preprocess_text, method='snowballstemming')


In [6]:
train_df.dropna(subset='Review',inplace=True)
train_df.dropna(subset='ssprocessing_text',inplace=True)
# train_df.dropna(subset='psprocessing_text',inplace=True)
# train_df.dropna(subset='lemprocessing_text',inplace=True)
train_df = train_df[train_df['Review'].str.len() >= 3]
train_df = train_df[train_df['ssprocessing_text'].str.len() >= 3]
# train_df = train_df[train_df['psprocessing_text'].str.len() >= 3]
# train_df = train_df[train_df['lemprocessing_text'].str.len() >= 3]
train_df

C:\Users\ishaq\AppData\Local\Temp\ipykernel_10648\1420501431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset='Review',inplace=True)
C:\Users\ishaq\AppData\Local\Temp\ipykernel_10648\1420501431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.dropna(subset='ssprocessing_text',inplace=True)


,overall,Review,ssprocessing_text
0,5,I love these glitter pens. They sparkle deligh...,love pen sparkl delight brilliant colour even ...
1,5,It works well with my machine. I use mostly c...,work well most cone
2,5,"This is a great assortment of colors, though t...",great assort color though lot pink mix still c...
3,5,Just what I was looking for.,just
4,5,I make 400 birds for the hospital each month.,400 bird hospit month
...,...,...,...
370858,5,I love these dies they make great backgrounds ...,love die great background card
370859,5,I love Darice embossing folders. Darcie's fol...,love daric emboss folder darci folder veri rea...
370860,5,I ordered these to add to my Earthy Markers th...,add earthi previous want brighter primari colo...
370861,4,Made perfect off white color by blending a tin...,perfect white blend tini amount yellow 2 white...


# Model Testing


### Raw Data


In [ ]:
# Initially was testing with smaller sample
train_subset = train_df[:]

# Fast text parameters 
# I use fastText because its faster and can sometimes perserve the under
vector_size = 50 # Represent text in 50 dimensions 
window = 15 # maximum text window to consider [wordn..., wordn+15]
min_count = 10  # word must appear 15 times before model considers it

train_data, test_data = train_test_split(train_subset, test_size=0.2, random_state=42)
# Tokenize
sentences = [text.split() for text in train_data['Review']]
# Create embeddings
model_fasttext = FastText(sentences, vector_size=vector_size, window=window, min_count=min_count)
# Count vectorizer with 1->3 ngrams
count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
count_vectorizer.fit(train_data['Review'])
# generate training and test data ngrams and counts according the the count vectorizor specified
X_train_count = count_vectorizer.transform(train_data['Review'])
X_test_count = count_vectorizer.transform(test_data['Review'])

In [ ]:
# Initially was testing with smaller sample
train_subset = train_df[:]

# Fast text parameters 
# I use fastText because its faster and can sometimes perserve the under
vector_size = 50 # Represent text in 50 dimensions 
window = 15 # maximum text window to consider [wordn..., wordn+15]
min_count = 10  # word must appear 15 times before model considers it

train_data, test_data = train_test_split(train_subset, test_size=0.2, random_state=42)
# Tokenize
sentences = [text.split() for text in train_data['Review']]
# Create embeddings
model_fasttext = FastText(sentences, vector_size=vector_size, window=window, min_count=min_count)
# Count vectorizer with 1->3 ngrams
count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
count_vectorizer.fit(train_data['Review'])
# generate training and test data ngrams and counts according the the count vectorizor specified
X_train_count = count_vectorizer.transform(train_data['Review'])
X_test_count = count_vectorizer.transform(test_data['Review'])

In [ ]:
# Function to generate word embeddings for data in batches
def generate_embeddings_batch(data, model, batch_size=1000):
    embeddings = []
    # Data slicing logic
    num_batches = (len(data) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size : (i + 1) * batch_size]
        batch_embeddings = []
        for text in batch_data:
            tokens = text.split()
            # For each token get embedding using embedding model (fastText) 
            token_embeddings = [model.wv[word] for word in tokens if word in model.wv]
            if token_embeddings:
                batch_embeddings.append(np.mean(token_embeddings, axis=0))
            else:
                # If token embedding is an invalid append zeros
                batch_embeddings.append(np.zeros(model.vector_size))
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [ ]:
# Embeddings on unprocessed review data using the fastText Model
X_train_embeddings = generate_embeddings_batch(train_data['Review'], model_fasttext)
X_test_embeddings = generate_embeddings_batch(test_data['Review'], model_fasttext)

X_train_count_sparse = X_train_count.astype(np.float32)
X_test_count_sparse = X_test_count.astype(np.float32)
X_train_embeddings_sparse = X_train_embeddings.astype(np.float32)
X_test_embeddings_sparse = X_test_embeddings.astype(np.float32)

# Stack sparse matrics of the Count vectorizor and embedding
X_train_combined = hstack([X_train_embeddings_sparse, X_train_count_sparse])
X_test_combined = hstack([X_test_embeddings_sparse, X_test_count_sparse])

In [ ]:
classifier = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1100)
classifier.fit(X_train_combined, train_data['overall'])

In [ ]:
y_pred = classifier.predict(X_test_combined)
report = classification_report(test_data['overall'], y_pred, output_dict=True)

results = pd.DataFrame(columns=['Processing', 'N-gram Range', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
results.loc[len(results)] = ['Logistic Regression (FastText + TF-IDF)', (1, 3), 
                             report['accuracy'], report['weighted avg']['precision'], 
                             report['weighted avg']['recall'], report['weighted avg']['f1-score']]

results

In [ ]:
test_df['Review'].fillna('',inplace=True)
test_sentences = [text.split() for text in test_df['Review']]

test_embeddings = generate_embeddings_batch(test_df['Review'], model_fasttext)

X_test_count = count_vectorizer.transform(test_df['Review'])

X_test_combined = hstack([test_embeddings.astype(np.float32), X_test_count.astype(np.float32)])

y_pred_test = classifier.predict(X_test_combined)

test_df['overall'] = y_pred_test
test_df['overall'].drop(columns=['Review'])
test_df.to_csv('raw_predictions.csv', index=False)


### Snowball Preprocessed ((Best))


In [ ]:
vector_size = 50
window = 15
min_count = 10

train_subset = train_df[:]

train_data, test_data = train_test_split(train_subset, test_size=0.2, random_state=42)

sentences = [text.split() for text in train_data['ssprocessing_text']]
model_fasttext = FastText(sentences, vector_size=vector_size, window=window, min_count=min_count)

count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
count_vectorizer.fit(train_data['ssprocessing_text'])

X_train_count = count_vectorizer.transform(train_data['ssprocessing_text'])
X_test_count = count_vectorizer.transform(test_data['ssprocessing_text'])

In [ ]:

def generate_embeddings_batch(data, model, batch_size=1000):
    embeddings = []
    num_batches = (len(data) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size : (i + 1) * batch_size]
        batch_embeddings = []
        for text in batch_data:
            tokens = text.split()
            token_embeddings = [model.wv[word] for word in tokens if word in model.wv]
            if token_embeddings:
                batch_embeddings.append(np.mean(token_embeddings, axis=0))
            else:
                batch_embeddings.append(np.zeros(model.vector_size))
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [ ]:
X_train_embeddings = generate_embeddings_batch(train_data['ssprocessing_text'], model_fasttext)
X_test_embeddings = generate_embeddings_batch(test_data['ssprocessing_text'], model_fasttext)

X_train_count_sparse = X_train_count.astype(np.float32)
X_test_count_sparse = X_test_count.astype(np.float32)
X_train_embeddings_sparse = X_train_embeddings.astype(np.float32)
X_test_embeddings_sparse = X_test_embeddings.astype(np.float32)

X_train_combined = hstack([X_train_embeddings_sparse, X_train_count_sparse])
X_test_combined = hstack([X_test_embeddings_sparse, X_test_count_sparse])

In [ ]:
classifier = LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1100)
classifier.fit(X_train_combined, train_data['overall'])

In [ ]:
y_pred = classifier.predict(X_test_combined)
report = classification_report(test_data['overall'], y_pred, output_dict=True)

results = pd.DataFrame(columns=['Processing', 'N-gram Range', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
results.loc[len(results)] = ['Logistic Regression (FastText + TF-IDF)', (1, 3), 
                             report['accuracy'], report['weighted avg']['precision'], 
                             report['weighted avg']['recall'], report['weighted avg']['f1-score']]

results

In [ ]:
test_df['ssprocessing_text'] = test_df['Review'].apply(preprocess_text)
test_df['ssprocessing_text'].fillna('',inplace=True)

test_embeddings = generate_embeddings_batch(test_df['ssprocessing_text'], model_fasttext)
X_test_count = count_vectorizer.transform(test_df['ssprocessing_text'])
X_test_combined = hstack([test_embeddings, X_test_count.astype(np.float32)])
y_pred_test = classifier.predict(X_test_combined)

test_df['overall'] = y_pred_test

test_df.to_csv('snowball_preprocessed_predictions.csv', index=False)

### RandomForest Preprocessed 79.3


In [7]:
vector_size = 50
window = 15
min_count = 10

train_subset = train_df[:]

train_data, test_data = train_test_split(train_subset, test_size=0.2, random_state=42)

sentences = [text.split() for text in train_data['ssprocessing_text']]
model_fasttext = FastText(sentences, vector_size=vector_size, window=window, min_count=min_count)

count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
count_vectorizer.fit(train_data['ssprocessing_text'])

X_train_count = count_vectorizer.transform(train_data['ssprocessing_text'])
X_test_count = count_vectorizer.transform(test_data['ssprocessing_text'])

In [8]:

def generate_embeddings_batch(data, model, batch_size=1000):
    embeddings = []
    num_batches = (len(data) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size : (i + 1) * batch_size]
        batch_embeddings = []
        for text in batch_data:
            tokens = text.split()
            token_embeddings = [model.wv[word] for word in tokens if word in model.wv]
            if token_embeddings:
                batch_embeddings.append(np.mean(token_embeddings, axis=0))
            else:
                batch_embeddings.append(np.zeros(model.vector_size))
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [9]:
X_train_embeddings = generate_embeddings_batch(train_data['ssprocessing_text'], model_fasttext)
X_test_embeddings = generate_embeddings_batch(test_data['ssprocessing_text'], model_fasttext)

X_train_count_sparse = X_train_count.astype(np.float32)
X_test_count_sparse = X_test_count.astype(np.float32)
X_train_embeddings_sparse = X_train_embeddings.astype(np.float32)
X_test_embeddings_sparse = X_test_embeddings.astype(np.float32)

X_train_combined = hstack([X_train_embeddings_sparse, X_train_count_sparse])
X_test_combined = hstack([X_test_embeddings_sparse, X_test_count_sparse])

In [10]:
classifier = RandomForestClassifier(n_estimators=5, random_state=42)
classifier.fit(X_train_combined, train_data['overall'])

RandomForestClassifier(n_estimators=5, random_state=42)

In [11]:
y_pred = classifier.predict(X_test_combined)
report = classification_report(test_data['overall'], y_pred, output_dict=True)

results = pd.DataFrame(columns=['Processing', 'N-gram Range', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
results.loc[len(results)] = ['RandomForestClassifier + TF-IDF)', (1, 3), 
                             report['accuracy'], report['weighted avg']['precision'], 
                             report['weighted avg']['recall'], report['weighted avg']['f1-score']]

results

,Processing,N-gram Range,Accuracy,Precision,Recall,F1-score
0,RandomForestClassifier + TF-IDF),"(1, 3)",0.793428,0.752442,0.793428,0.75303


In [12]:
test_df['ssprocessing_text'] = test_df['Review'].apply(preprocess_text)
test_df['ssprocessing_text'].fillna('',inplace=True)

test_embeddings = generate_embeddings_batch(test_df['ssprocessing_text'], model_fasttext)
X_test_count = count_vectorizer.transform(test_df['ssprocessing_text'])
X_test_combined = hstack([test_embeddings, X_test_count.astype(np.float32)])
y_pred_test = classifier.predict(X_test_combined)

test_df['overall'] = y_pred_test

test_df.to_csv('RF_snowball_preprocessed_predictions.csv', index=False)

### GradientBoostingClassifier Preprocessed


In [13]:
vector_size = 50
window = 15
min_count = 10

train_subset = train_df[:]

train_data, test_data = train_test_split(train_subset, test_size=0.2, random_state=42)

sentences = [text.split() for text in train_data['ssprocessing_text']]
model_fasttext = FastText(sentences, vector_size=vector_size, window=window, min_count=min_count)

count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
count_vectorizer.fit(train_data['ssprocessing_text'])

X_train_count = count_vectorizer.transform(train_data['ssprocessing_text'])
X_test_count = count_vectorizer.transform(test_data['ssprocessing_text'])

In [14]:

def generate_embeddings_batch(data, model, batch_size=1000):
    embeddings = []
    num_batches = (len(data) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size : (i + 1) * batch_size]
        batch_embeddings = []
        for text in batch_data:
            tokens = text.split()
            token_embeddings = [model.wv[word] for word in tokens if word in model.wv]
            if token_embeddings:
                batch_embeddings.append(np.mean(token_embeddings, axis=0))
            else:
                batch_embeddings.append(np.zeros(model.vector_size))
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [15]:
X_train_embeddings = generate_embeddings_batch(train_data['ssprocessing_text'], model_fasttext)
X_test_embeddings = generate_embeddings_batch(test_data['ssprocessing_text'], model_fasttext)

X_train_count_sparse = X_train_count.astype(np.float32)
X_test_count_sparse = X_test_count.astype(np.float32)
X_train_embeddings_sparse = X_train_embeddings.astype(np.float32)
X_test_embeddings_sparse = X_test_embeddings.astype(np.float32)

X_train_combined = hstack([X_train_embeddings_sparse, X_train_count_sparse])
X_test_combined = hstack([X_test_embeddings_sparse, X_test_count_sparse])

In [16]:
classifier = GradientBoostingClassifier(n_estimators=10, random_state=42)
classifier.fit(X_train_combined, train_data['overall'])

GradientBoostingClassifier(n_estimators=10, random_state=42)

In [17]:
y_pred = classifier.predict(X_test_combined)
report = classification_report(test_data['overall'], y_pred, output_dict=True)

results = pd.DataFrame(columns=['Processing', 'N-gram Range', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
results.loc[len(results)] = ['GradientBoostingClassifier (FastText + TF-IDF)', (1, 3), 
                             report['accuracy'], report['weighted avg']['precision'], 
                             report['weighted avg']['recall'], report['weighted avg']['f1-score']]

results

,Processing,N-gram Range,Accuracy,Precision,Recall,F1-score
0,GradientBoostingClassifier (FastText + TF-IDF),"(1, 3)",0.764469,0.718365,0.764469,0.665023


In [18]:
test_df['ssprocessing_text'] = test_df['Review'].apply(preprocess_text)
test_df['ssprocessing_text'].fillna('',inplace=True)

test_embeddings = generate_embeddings_batch(test_df['ssprocessing_text'], model_fasttext)
X_test_count = count_vectorizer.transform(test_df['ssprocessing_text'])
X_test_combined = hstack([test_embeddings, X_test_count.astype(np.float32)])
y_pred_test = classifier.predict(X_test_combined)

test_df['overall'] = y_pred_test

test_df.to_csv('XGB_snowball_preprocessed_predictions.csv', index=False)

In [19]:
train_data

,overall,Review,ssprocessing_text
115196,5,"Very nice quality, good variation in color in ...",veri nice good variat strand
161295,5,"Love it, Great Item! Great service!",love great great
100745,5,good,good
1488,5,I love the bag. Now I can store the Zutter Bin...,love bag now can zutter accompani tool
4063,5,"Like the Ice resin over other resins, crisper ...",like ice resin resin crisper cleaner great thank
...,...,...,...
261496,3,Handle is good.\nNeedle point and edges is dul...,handl good needl point edg dull needl too thin...
369094,4,Excellent price point. Consistent color and te...,excel point consist textur quick
133109,5,i love this yarn !,love yarn
148194,5,Thank you!,thank


### SVC Preprocessed


In [ ]:
vector_size = 50
window = 15
min_count = 10

train_subset = train_df[:]

train_data, test_data = train_test_split(train_subset, test_size=0.2, random_state=42)

sentences = [text.split() for text in train_data['ssprocessing_text']]
model_fasttext = FastText(sentences, vector_size=vector_size, window=window, min_count=min_count)

count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
count_vectorizer.fit(train_data['ssprocessing_text'])

X_train_count = count_vectorizer.transform(train_data['ssprocessing_text'])
X_test_count = count_vectorizer.transform(test_data['ssprocessing_text'])

In [ ]:

def generate_embeddings_batch(data, model, batch_size=1000):
    embeddings = []
    num_batches = (len(data) + batch_size - 1) // batch_size
    for i in range(num_batches):
        batch_data = data.iloc[i * batch_size : (i + 1) * batch_size]
        batch_embeddings = []
        for text in batch_data:
            tokens = text.split()
            token_embeddings = [model.wv[word] for word in tokens if word in model.wv]
            if token_embeddings:
                batch_embeddings.append(np.mean(token_embeddings, axis=0))
            else:
                batch_embeddings.append(np.zeros(model.vector_size))
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

In [ ]:
X_train_embeddings = generate_embeddings_batch(train_data['ssprocessing_text'], model_fasttext)
X_test_embeddings = generate_embeddings_batch(test_data['ssprocessing_text'], model_fasttext)

X_train_count_sparse = X_train_count.astype(np.float32)
X_test_count_sparse = X_test_count.astype(np.float32)
X_train_embeddings_sparse = X_train_embeddings.astype(np.float32)
X_test_embeddings_sparse = X_test_embeddings.astype(np.float32)

X_train_combined = hstack([X_train_embeddings_sparse, X_train_count_sparse])
X_test_combined = hstack([X_test_embeddings_sparse, X_test_count_sparse])

In [ ]:
classifier = SVC(max_iter=1000, kernel='rbf',C=1.0)
classifier.fit(X_train_combined, train_data['overall'])
# 

c:\Users\ishaq\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(max_iter=1000)

In [ ]:
y_pred = classifier.predict(X_test_combined)
report = classification_report(test_data['overall'], y_pred, output_dict=True)

results = pd.DataFrame(columns=['Processing', 'N-gram Range', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
results.loc[len(results)] = ['GradientBoostingClassifier (FastText + TF-IDF)', (1, 3), 
                             report['accuracy'], report['weighted avg']['precision'], 
                             report['weighted avg']['recall'], report['weighted avg']['f1-score']]

results

c:\Users\ishaq\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ishaq\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ishaq\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

,Processing,N-gram Range,Accuracy,Precision,Recall,F1-score
0,GradientBoostingClassifier (FastText + TF-IDF),"(1, 3)",0.768,0.72048,0.768,0.672661


In [ ]:
test_df['ssprocessing_text'] = test_df['Review'].apply(preprocess_text)
test_df['ssprocessing_text'].fillna('',inplace=True)

test_embeddings = generate_embeddings_batch(test_df['ssprocessing_text'], model_fasttext)
X_test_count = count_vectorizer.transform(test_df['ssprocessing_text'])
X_test_combined = hstack([test_embeddings, X_test_count.astype(np.float32)])
y_pred_test = classifier.predict(X_test_combined)

test_df['overall'] = y_pred_test

test_df.to_csv('SVC_snowball_preprocessed_predictions.csv', index=False)

KeyboardInterrupt: 

## LSTM Preprocessed


In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D, Concatenate, Input
from keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.utils import pad_sequences,to_categorical
from keras.layers import Concatenate


train_subset = train_df[:10000]

train_data, test_data = train_test_split(train_subset, test_size=0.2, random_state=42)

max_words = 50000 
max_length = 5 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data['ssprocessing_text'])

X_train_seq = tokenizer.texts_to_sequences(train_data['ssprocessing_text'])
X_test_seq = tokenizer.texts_to_sequences(test_data['ssprocessing_text'])

X_train_seq = pad_sequences(X_train_seq, maxlen=max_length)
X_test_seq = pad_sequences(X_test_seq, maxlen=max_length)

count_vectorizer = CountVectorizer(ngram_range=(1, 3)) 
X_train_count = count_vectorizer.fit_transform(train_data['ssprocessing_text']).toarray()
X_test_count = count_vectorizer.transform(test_data['ssprocessing_text']).toarray()

input_seq = Input(shape=(max_length,))
input_count = Input(shape=(X_train_count.shape[1],))

embedding = Embedding(input_dim=max_words, output_dim=32, input_length=max_length)(input_seq)
lstm = LSTM(150, dropout=0.2, recurrent_dropout=0.2)(embedding)
merged = Concatenate()([lstm, input_count])
output = Dense(5, activation='softmax')(merged)
model = Model(inputs=[input_seq, input_count], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])

target_train = to_categorical(train_data['overall']-1, num_classes=5).reshape(-1, 5)
target_test = to_categorical(test_data['overall']-1, num_classes=5).reshape(-1, 5)

history = model.fit([X_train_seq, X_train_count], target_train, epochs=10, batch_size=128, validation_data=([X_test_seq, X_test_count], target_test))


c:\Users\ishaq\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
63/63 [==============================] - 24s 320ms/step - loss: 1.0492 - accuracy: 0.7515 - val_loss: 0.7803 - val_accuracy: 0.7650
Epoch 2/10
63/63 [==============================] - 21s 339ms/step - loss: 0.5762 - accuracy: 0.8008 - val_loss: 0.7270 - val_accuracy: 0.7675
Epoch 3/10
63/63 [==============================] - 20s 325ms/step - loss: 0.4509 - accuracy: 0.8575 - val_loss: 0.7209 - val_accuracy: 0.7665
Epoch 4/10
63/63 [==============================] - 28s 441ms/step - loss: 0.3754 - accuracy: 0.8934 - val_loss: 0.7392 - val_accuracy: 0.7675
Epoch 5/10
63/63 [==============================] - 20s 318ms/step - loss: 0.3246 - accuracy: 0.9121 - val_loss: 0.7171 - val_accuracy: 0.7700
Epoch 6/10
63/63 [==============================] - 20s 326ms/step - loss: 0.2879 - accuracy: 0.9291 - val_loss: 0.7424 - val_accuracy: 0.7690
Epoch 7/10
63/63 [==============================] - 23s 366ms/step - loss: 0.2593 - accuracy: 0.9361 - val_loss: 0.7503 - val_accuracy: 0.7630

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict([X_test_seq, X_test_count])
y_pred_classes = np.argmax(y_pred, axis=1)
report = classification_report(np.argmax(test_data['overall'], axis=1), y_pred_classes, output_dict=True)

results = pd.DataFrame(columns=['Processing', 'Accuracy', 'Precision', 'Recall', 'F1-score'])
results.loc[len(results)] = ['LSTM + CountVectorizer', 
                             report['accuracy'], 
                             report['weighted avg']['precision'], 
                             report['weighted avg']['recall'], 
                             report['weighted avg']['f1-score']]

results

In [ ]:
test_df['ssprocessing_text'] = test_df['Review'].apply(preprocess_text)
test_df['ssprocessing_text'].fillna('',inplace=True)

test_embeddings = generate_embeddings_batch(test_df['ssprocessing_text'], model_fasttext)
X_test_count = count_vectorizer.transform(test_df['ssprocessing_text'])
X_test_combined = hstack([test_embeddings, X_test_count.astype(np.float32)])
y_pred_test = classifier.predict(X_test_combined)

test_df['overall'] = y_pred_test

test_df.to_csv('LSTM_snowball_preprocessed_predictions.csv', index=False)